In [2]:
import json
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import cv2
import PIL
from PIL import Image
from PIL import ImageFont, ImageDraw
from tqdm import tqdm

# Preparing metadata for the dataset for the NetVLAD

In [11]:
root_path = '/media/cds-s/data/Datasets/Habitat/extracted_HPointLoc/1LXtFkjw3qL_point0/'

In [12]:
data = {}
data['dbImage'] = []
data['qImage'] = []

In [20]:
exts = ['*.jpg', '*.png', '*.jpeg', '*.JPG', '*.PNG']
paths = []
for ext in exts:
    paths += list(Path(root_path).glob('**/'+ext))

In [19]:
paths[0].relative_to(Path(root_path))

PosixPath('images/1LXtFkjw3qL_point13_query_0013.png')

In [8]:
for filename in sorted(os.listdir(root_path)):
    if filename.find('.png') == -1:
        continue
    if filename.find('database') != -1:
        data['dbImage'].append(os.path.join(filename))
    if filename.find('query') != -1:
        data['qImage'].append(os.path.join(filename))
data['numDb'] = len(data['dbImage'])
data['numQ'] = len(data['qImage'])

In [9]:
with open('/media/cds-s/data/Datasets/Habitat/metadata.json', 'w') as fp:
    json.dump(data, fp)

# Visualizing results

In [80]:
def read_image(path):
    image = cv2.imread(str(path))
    if len(image.shape) == 3:
        image = image[:, :, ::-1]
    return image

In [81]:
def plot_images(imgs, titles=None, cmaps='gray', dpi=100, pad=.5,
                adaptive=True):
    """Plot a set of images horizontally.
    Args:
        imgs: a list of NumPy or PyTorch images, RGB (H, W, 3) or mono (H, W).
        titles: a list of strings, as titles for each image.
        cmaps: colormaps for monochrome images.
        adaptive: whether the figure size should fit the image aspect ratios.
    """
    n = len(imgs)
    if not isinstance(cmaps, (list, tuple)):
        cmaps = [cmaps] * n

    if adaptive:
        ratios = [i.shape[1] / i.shape[0] for i in imgs]  # W / H
    else:
        ratios = [4/3] * n
    figsize = [sum(ratios)*4.5, 4.5]
    fig, ax = plt.subplots(
        1, n, figsize=figsize, dpi=dpi, gridspec_kw={'width_ratios': ratios})
    if n == 1:
        ax = [ax]
    for i in range(n):
        ax[i].imshow(imgs[i], cmap=plt.get_cmap(cmaps[i]))
        ax[i].get_yaxis().set_ticks([])
        ax[i].get_xaxis().set_ticks([])
        ax[i].set_axis_off()
        for spine in ax[i].spines.values():  # remove frame
            spine.set_visible(False)
        if titles:
            ax[i].set_title(titles[i])
    fig.tight_layout(pad=pad)

In [3]:
input_json_file = '/media/cds-s/data/Datasets/Habitat/results.json'

In [4]:
with open(input_json_file) as json_file:
    results = json.load(json_file)

In [19]:
pwd

'/home/cds-s'

In [20]:
!mkdir matches

mkdir: cannot create directory ‘matches’: File exists


In [24]:
count = 0
for query_image_filename, prediction_filename in tqdm(results.items()):
    count = count + 1
    
    query_image_filename = os.path.relpath(query_image_filename, '/datasets')
    prediction_filename_1, prediction_filename_2, prediction_filename_3 = prediction_filename[:3]
    
    prediction_filename_1 = os.path.relpath(prediction_filename_1, '/datasets')
    prediction_filename_2 = os.path.relpath(prediction_filename_2, '/datasets')
    prediction_filename_3 = os.path.relpath(prediction_filename_3, '/datasets')
    
    query_image_filename = os.path.join('/media/cds-s/data/Datasets', query_image_filename)
    prediction_filename_1 = os.path.join('/media/cds-s/data/Datasets', prediction_filename_1)
    prediction_filename_2 = os.path.join('/media/cds-s/data/Datasets', prediction_filename_2)
    prediction_filename_3 = os.path.join('/media/cds-s/data/Datasets', prediction_filename_3)
    
    image1 = Image.open(query_image_filename)
    image2 = Image.open(prediction_filename_1)
    image3 = Image.open(prediction_filename_2)
    image4 = Image.open(prediction_filename_3)
    
    image1_size = image1.size
    
    new_image = Image.new('RGB',(4*image1_size[0], image1_size[1]), (250,250,250))
    new_image.paste(image1,(0,0))
    new_image.paste(image2,(image1_size[0],0))
    new_image.paste(image3,(2*image1_size[0],0))
    new_image.paste(image4,(3*image1_size[0],0))
    
    draw = ImageDraw.Draw(new_image)
    
    font = ImageFont.truetype('/usr/share/fonts/truetype/liberation/LiberationMono-Bold.ttf', 12, encoding="unic")
    
#     query_image = read_image(query_image_filename)
#     prediction_image = read_image(prediction_filename)
#     plot_images([query_image, prediction_image])

    draw.text((0, 0),query_image_filename.split('/')[-1],(255,255,255),font=font)
    draw.text((image1.size[0], 0),"{}".format(prediction_filename_1.split('/')[-1]),(255,255,255),font=font)
    draw.text((2*image1.size[0], 0),"{}".format(prediction_filename_2.split('/')[-1]),(255,255,255),font=font)
    draw.text((3*image1.size[0], 0),"{}".format(prediction_filename_3.split('/')[-1]),(255,255,255),font=font)
    new_image.save('matches/{}.png'.format(query_image_filename.split('/')[-1]),'PNG')

#     if count == 20:
#         break

100%|██████████| 950/950 [00:47<00:00, 19.81it/s]


In [86]:
query_image_filename

'/media/cds-s/data/Datasets/Habitat/1LXtFkjw3qL_point0/images/1LXtFkjw3qL_point0/around_0009.png'

In [89]:
os.path.join('/media/cds-s/data/Datasets', 'fgdfg')

'/media/cds-s/data/Datasets/fgdfg'

In [121]:
pwd

'/home/cds-s'

In [13]:
data = np.load('/media/cds-s/data/Datasets/Habitat/HF_Net_output/glb/1LXtFkjw3qL_point0_database_0000.npy')

In [14]:
data

array([ 0.01135713, -0.00560202, -0.00253877, ...,  0.01415169,
       -0.01611472, -0.01287946], dtype=float32)

In [15]:
data.shape

(4096,)

In [16]:
np.sum(data**2)**0.5

1.0000001192092824